In [1]:
from graph_tool.all import *
from graph_tool import topology
from graph_tool import draw

%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

In [2]:
path_to_file = 'wiki-Vote.txt'

# this way of loading the graph does not work as it mapped the original vertex index label to its own

In [ ]:
%%time
G_dir = load_graph_from_csv(path_to_file, directed=True,csv_options={'delimiter': '\t'})
G_undir = G_dir
G_undir.set_directed(False)

#G_undir = load_graph_from_csv(path_to_file, directed=False,csv_options={'delimiter': '\t'})

# manually load the graph

In [3]:
import csv

g = Graph()

#e_weight = g.new_edge_property('float')

f_network = open(path_to_file, 'r')

reader_network = csv.reader(f_network, delimiter='\t', skipinitialspace=True)

for edge in reader_network:
    e = g.add_edge(int(edge[0]), int(edge[1]))
    #e_weight[e] = float(1)


f_network.close()

g.set_directed(True)

In [4]:
%%time
l = topology.label_largest_component(g, directed=True)
LSCC = GraphView(g, vfilt=l)

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 43.9 ms


In [5]:
%%time
w = topology.label_largest_component(g, directed=False)
LWCC = GraphView(g, vfilt=w) 

CPU times: user 0 ns, sys: 10 ms, total: 10 ms
Wall time: 3.72 ms


In [6]:
print('LSCC edges: \t', LSCC.num_edges())
print('LSCC nodes: \t', LSCC.num_vertices())

print('LWCC edges: \t',LWCC.num_edges())
print('LWCC nodes: \t',LWCC.num_vertices())

LSCC edges: 	 39456
LSCC nodes: 	 1300
LWCC edges: 	 103663
LWCC nodes: 	 7066


## calulate distances for LSCC

In [9]:
%%time
distances = topology.shortest_distance(LSCC, directed=True)

CPU times: user 750 ms, sys: 80 ms, total: 830 ms
Wall time: 342 ms


In [12]:
%%time
dist_dir = []
LSCC_vertices = LSCC.get_vertices()
counter = 0 
for v1 in LSCC_vertices:
    for v2 in LSCC_vertices:
        if v1 != v2:
            dist_dir.append(distances[v1][v2])
        if (counter%10000 == 0):
            print(counter)
        counter = counter + 1


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [13]:
print('median distance:\t',np.percentile(dist_dir,50))
print('mean distance:\t',np.mean(dist_dir))
print('diameter:\t',np.max(dist_dir))
print('effective diameter:\t',np.percentile(dist_dir,90))

median distance:	 3.0
mean distance:	 2.87928288032
diameter:	 9
effective diameter:	 4.0


## calculate distances for LWCC

In [23]:
%%time
distances_undir = topology.shortest_distance(LWCC, directed=False)

CPU times: user 11.2 s, sys: 380 ms, total: 11.6 s
Wall time: 3.14 s


In [ ]:
%%time
dist_undir = []
LWCC_vertices = LWCC.get_vertices()
counter = 0
for v1 in LWCC_vertices:
    for v2 in LWCC_vertices:
        if v1 != v2:
            dist_undir.append(distances_undir[v1][v2])
        if (counter%100000 == 0):
            print(np.round(counter/10**6,1), 'mio')
        counter = counter + 1

0.0 mio
0.1 mio
0.2 mio
0.3 mio
0.4 mio
0.5 mio
0.6 mio
0.7 mio
0.8 mio
0.9 mio
1.0 mio
1.1 mio
1.2 mio
1.3 mio
1.4 mio
1.5 mio
1.6 mio
1.7 mio
1.8 mio
1.9 mio
2.0 mio
2.1 mio
2.2 mio
2.3 mio
2.4 mio
2.5 mio
2.6 mio
2.7 mio
2.8 mio
2.9 mio
3.0 mio
3.1 mio
3.2 mio
3.3 mio
3.4 mio
3.5 mio
3.6 mio
3.7 mio
3.8 mio
3.9 mio
4.0 mio
4.1 mio
4.2 mio
4.3 mio
4.4 mio
4.5 mio
4.6 mio
4.7 mio
4.8 mio
4.9 mio
5.0 mio
5.1 mio
5.2 mio
5.3 mio
5.4 mio
5.5 mio
5.6 mio
5.7 mio
5.8 mio
5.9 mio
6.0 mio
6.1 mio
6.2 mio
6.3 mio
6.4 mio
6.5 mio
6.6 mio
6.7 mio
6.8 mio
6.9 mio
7.0 mio
7.1 mio
7.2 mio
7.3 mio
7.4 mio
7.5 mio
7.6 mio
7.7 mio
7.8 mio
7.9 mio
8.0 mio
8.1 mio
8.2 mio
8.3 mio
8.4 mio
8.5 mio
8.6 mio
8.7 mio
8.8 mio
8.9 mio
9.0 mio
9.1 mio
9.2 mio
9.3 mio
9.4 mio
9.5 mio
9.6 mio
9.7 mio
9.8 mio
9.9 mio
10.0 mio
10.1 mio
10.2 mio
10.3 mio
10.4 mio
10.5 mio
10.6 mio
10.7 mio
10.8 mio
10.9 mio
11.0 mio
11.1 mio
11.2 mio
11.3 mio
11.4 mio
11.5 mio
11.6 mio
11.7 mio
11.8 mio
11.9 mio
12.0 mio
12.1 mio
12

In [ ]:
print('median distance:\t',np.percentile(dist_undir,50))
print('mean distance:\t',np.mean(dist_undir))
print('diameter:\t',np.max(dist_undir))
print('effective diameter:\t',np.percentile(dist_undir,90))

In [ ]:
1+1